# Sampling Rig Contractors - Sampling without max - KNN

Here the thought process is similar to the sampling without replacement, but we understand that amplifying the smaller contractors may negatively impact the over all model. So instead of taking the total number of entries for each contractor and dividing equally, we will take the sum of the lesser 4 contractors and use that to come up with the number of entries to use.

In [1]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Establishing Path

In [2]:
path = "../SHARED/"

## Recompletion

In [3]:
# Reading in data
data_rcompl = pd.read_excel(path + "RF_imputed_TO_TACC_RCOMPL.xlsx")

In [4]:
# Getting the unique contractors
unique_rigs = data_rcompl["CONTRACTOR"].unique()

In [5]:
# Get the totals of the contractors
total_entries = []

for i in unique_rigs:
    temp_pd = data_rcompl[data_rcompl["CONTRACTOR"] == i]
    print(i, " has ", len(temp_pd), " elements.")
    total_entries.append(len(temp_pd))

2  has  805  elements.
0  has  259  elements.
4  has  22  elements.
3  has  403  elements.
1  has  165  elements.


In [6]:
total_entries

[805, 259, 22, 403, 165]

In [7]:
# Here we will be summing everything up but before we get our samples
# we will remove the largest number of elements and essentially ignore it
# from counting

sum_rcompl = 0

for i in total_entries:
    sum_rcompl += i

tot_samples = (sum_rcompl-805) // 4

In [8]:
tot_samples

212

In [9]:
# knowing our new sample size we go ahead and sampel with replacement
new_rcompl = pd.DataFrame(columns = data_rcompl.columns)

for i in range(0, len(unique_rigs)):
    temp_pd = data_rcompl[data_rcompl["CONTRACTOR"] == unique_rigs[i]]
    temp_pd = temp_pd.sample(n = tot_samples, replace = True)
    new_rcompl = new_rcompl.append(temp_pd, ignore_index=True)

In [10]:
new_rcompl

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,PROPPANT_IN_FORMATION,STG_NUMBER,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,STG_CLEAN_VOLUME_PUMPED,STG_SLURRY_VOLUME_PUMPED,BOTTOM_DEPTH_TVD,Rig_Duration (days),Stg_Duration (hrs),Stage_Length (meters)
0,32,0,0,33,20,0,2,0,33,42,...,452475,8,12262,18270,10284,13461,12405,69,3,6008
1,6,0,0,0,7,0,3,0,4,7,...,30000,24,12579,16912,5216,5242,11958,5,1,4333
2,15,0,0,21,4,1,2,0,20,26,...,306250,21,12802,18326,6060,6388,12140,11,1,5524
3,16,0,0,29,5,1,2,0,42,48,...,0,6,13491,17648,6195,6195,13014,5,1,4157
4,3,0,0,31,14,0,1,0,60,4,...,334380,2,17414,17560,6791,7157,11785,4,1,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,34,0,0,30,15,0,3,0,31,32,...,0,25,12730,17098,23818,23818,12262,6,6,4368
1056,0,0,0,51,15,1,0,0,52,51,...,328240,13,11975,14975,9933,10288,11650,6,2,3000
1057,17,0,0,56,14,0,2,0,55,52,...,375980,19,13139,18294,10219,10636,12875,6,4,5155
1058,17,0,0,56,9,1,2,0,55,52,...,369480,18,13139,18294,9805,10218,12875,6,2,5155


In [11]:
new_rcompl.to_excel(path + "RF_RCOMPL_RIG_SCALED_REPLACE_WO_MAX.xlsx")

## Stimulation stage

In [20]:
#Read in data
data_stmstg = pd.read_excel(path + "RF_imputed_TO_TACC_STM_STG.xlsx")

In [21]:
# Get the contractors
unique_rigs = data_stmstg["CONTRACTOR"].unique()

In [22]:
# see totals for the different contractors
total_entries = []

for i in unique_rigs:
    temp_pd = data_stmstg[data_stmstg["CONTRACTOR"] == i]
    print(i, " has ", len(temp_pd), " elements.")
    total_entries.append(len(temp_pd))

2  has  8790  elements.
0  has  3455  elements.
4  has  460  elements.
3  has  3825  elements.
1  has  1191  elements.


In [23]:
total_entries

[8790, 3455, 460, 3825, 1191]

In [24]:
# Here we will be summing everything up but before we get our samples
# we will remove the largest number of elements and essentially ignore it
# from counting

sum_stmstg = 0

for i in total_entries:
    sum_stmstg += i

print("total sum", sum_stmstg)
tot_samples = (sum_stmstg-8790) // 4

total sum 17721


In [25]:
tot_samples

2232

In [26]:
# knowing our new sample size we go ahead and sampel with replacement
new_stmstg = pd.DataFrame(columns = data_stmstg.columns)

for i in range(0, len(unique_rigs)):
    temp_pd = data_stmstg[data_stmstg["CONTRACTOR"] == unique_rigs[i]]
    temp_pd = temp_pd.sample(n = tot_samples, replace = True)
    new_stmstg = new_stmstg.append(temp_pd, ignore_index=True)

In [27]:
new_stmstg

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,Prod_Per_Stage,Errors_Per_Stage,UWI,UWI_SIDETRACK,REPORT_START_DATE_YEAR,STG_NUMBER,STG_CLEAN_VOLUME_PUMPED,STG_SLURRY_VOLUME_PUMPED,Rig_Duration (days),Stg_Duration (hrs)
0,46,1,0,93,61,0,2,0,123,161,...,27827.000000,8.000000,2019000085,0,2018,21,6515,6944,103,1
1,407,1,0,31,29,0,2,0,52,73,...,33836.111111,7.666667,2019000976,0,2017,36,11411,12034,93,3
2,46,1,0,93,61,0,2,0,123,161,...,27827.000000,7.000000,2019000085,0,2018,22,6435,6925,103,1
3,408,1,0,31,33,0,2,0,50,64,...,33836.111111,8.277778,2019000977,0,2017,3,10857,11473,138,4
4,232,2,0,141,57,1,2,0,192,186,...,42355.000000,9.000000,2019000788,0,2019,6,10276,10845,244,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11155,222,1,0,79,99,0,3,1,124,121,...,1176.288462,2.961538,2019000730,0,2017,7,6600,6919,6,1
11156,222,1,0,79,99,0,3,1,124,121,...,1176.288462,2.961538,2019000730,0,2017,3,6820,7148,6,2
11157,399,1,0,91,59,0,2,0,121,138,...,70460.000000,7.000000,2019000968,0,2017,4,9720,9771,165,2
11158,193,1,0,102,129,2,2,0,139,184,...,25406.000000,9.000000,2019000524,0,2017,19,14829,15615,96,3


In [20]:
# Export data
new_stmstg.to_excel(path + "RF_STMSTG_RIG_SCALED_REPLACE_WO_MAX.xlsx")